In [55]:
import pandas as pd
from tqdm import tqdm
import numpy as np
#from konlpy.tag import mecab
from gensim.models import Word2Vec
from scipy.spatial import distance
from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer

## 전처리

In [56]:
raw_df = pd.read_csv('theme_contents.csv', encoding = 'utf-8', index_col=0)
raw_df.dropna(inplace=True)

In [57]:
raw_df

,theme,contents
0,2차전지,한국무역보험공사(K-SURE)는 27일 이인호 무역보험공사 사장이 신종 코로나바이러...
1,2차전지,코스피 상장사 이엔플러스가 2차전지 효율을 극대화할 수 있는 '액상 그래핀' 개발...
2,2차전지,이엔플러스(074610)가 2차전지 효율을 극대화할 수 있는 액상 그래핀을 개발해...
3,2차전지,[[특징주]]이엔플러스가 세계 최초로 2차전지 효율을 극대화할 수 있는 액상 그래...
4,2차전지,2차전지 성장세로 관련 기업들이 함박웃음을 짓고 있다. 국내 배터리 3사의 연이은...
...,...,...
26695,희토류,(지디넷코리아=양태훈 기자)한국과 중국의 전문가들이 희토류 소재 분야에서 기술 협...
26696,희토류,테마가 강세다. 전일 대비 2.45% 상승세이다. 유니온머티리얼(047400)+7...
26697,희토류,"미중 무역분쟁 갈등이 고조되고 희토류 관련주들이 강세를 보이고 있는 가운데 상보,..."
26698,희토류,테마가 강세다. 전일 대비 2.31% 상승세이다. 티플랙스(081150)+7.39...


In [58]:
raw_df = raw_df[['theme', 'contents']]
raw_df['contents'] = raw_df['contents'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
raw_df.head()

,theme,contents
0,2차전지,한국무역보험공사는 일 이인호 무역보험공사 사장이 신종 코로나바이러스 감염증코로나 사...
1,2차전지,코스피 상장사 이엔플러스가 차전지 효율을 극대화할 수 있는 액상 그래핀 개발에 성...
2,2차전지,이엔플러스가 차전지 효율을 극대화할 수 있는 액상 그래핀을 개발해 오는 월부터 본...
3,2차전지,특징주이엔플러스가 세계 최초로 차전지 효율을 극대화할 수 있는 액상 그래핀을 개발...
4,2차전지,차전지 성장세로 관련 기업들이 함박웃음을 짓고 있다 국내 배터리 사의 연이은 증설...


In [59]:
tok_path = get_tokenizer()
sp  = SentencepieceTokenizer(tok_path)

using cached model


In [69]:
# tokenize data

raw_tokenized_data = []
for content in tqdm(raw_df['contents']):
    token = sp(content)
    #stop_words = ['으로', '로도', '지만', '에서', '려는', '하다', '은', '는', '이', '가', '을', '를', '수', '할']
    #for word in token:
        #if len(word) == 1:
            #stop_words.append(word)
    final_tokens = [word for word in token if word not in stop_words]
    raw_tokenized_data.append(final_tokens)

100%|██████████| 26700/26700 [00:27<00:00, 971.22it/s]


In [70]:
# make train_data

train_data = raw_df.copy()
train_data['tokens'] = pd.Series(raw_tokenized_data)

In [71]:
train_data.head()

,theme,contents,tokens
0,2차전지,한국무역보험공사는 일 이인호 무역보험공사 사장이 신종 코로나바이러스 감염증코로나 사...,"[▁한국, 무역, 보험, 공사, 는, ▁일, ▁이, 인, 호, ▁무역, 보험, 공사..."
1,2차전지,코스피 상장사 이엔플러스가 차전지 효율을 극대화할 수 있는 액상 그래핀 개발에 성...,"[▁코스피, ▁상장, 사, ▁이, 엔, 플러스, 가, ▁, 차전, 지, ▁, 효율,..."
2,2차전지,이엔플러스가 차전지 효율을 극대화할 수 있는 액상 그래핀을 개발해 오는 월부터 본...,"[▁이, 엔, 플러스, 가, ▁, 차전, 지, ▁, 효율, 을, ▁극대화, 할, ▁..."
3,2차전지,특징주이엔플러스가 세계 최초로 차전지 효율을 극대화할 수 있는 액상 그래핀을 개발...,"[▁, 특징주, 이, 엔, 플러스, 가, ▁세계, ▁최초로, ▁, 차전, 지, ▁,..."
4,2차전지,차전지 성장세로 관련 기업들이 함박웃음을 짓고 있다 국내 배터리 사의 연이은 증설...,"[▁, 차전, 지, ▁성장세, 로, ▁관련, ▁기업들, 이, ▁함, 박, 웃음, 을..."


## Model Train

In [73]:
news_tokens = train_data['tokens'].tolist()  # data input as list

# params
v_dimension = 300
v_window = 8

model = Word2Vec(sentences = news_tokens, size = v_dimension, window = v_window, min_count = 5, workers = 4, sg = 0)

In [74]:
model.wv.vectors.shape

(7207, 300)

In [75]:
# sentence vectors without normalization

def without_normal(tokens):
    vectors = []
    for token in tokens:
        init_v = np.array([0.0]*v_dimension)
        for word in token:
            word_vectors = model.wv
            if word in word_vectors.vocab:
                v = model.wv[word]
                init_v = init_v + v
        vectors.append(init_v)
    
    frame = { 'themes': train_data['theme'].tolist(), 'vectors': vectors }
    result = pd.DataFrame(frame) 
    
    return result

In [76]:
# theme vectors without normalization

def theme_without_normal(news_df):
    theme_list = []
    vector_list = []
    for theme in news_df['themes'].unique():
        temp_df = news_df.loc[news_df['themes'] == theme]
        add_v = np.array([0.0]*v_dimension)
        for vec in temp_df['vectors']:
            add_v  = add_v + vec
        theme_list.append(theme)
        vector_list.append(add_v)
        
    frame = { 'themes': theme_list, 'vectors': vector_list }
    result = pd.DataFrame(frame)
    
    return result

In [77]:
news_vectors_df = without_normal(train_data['tokens'])

In [78]:
news_vectors_df.head()

,themes,vectors
0,2차전지,"[-15.69059557619039, -64.53006806224585, 61.15..."
1,2차전지,"[-76.45044423209038, -213.35149738937616, 51.8..."
2,2차전지,"[-11.4459562265547, -150.51336188986897, 60.96..."
3,2차전지,"[-18.60038736835122, -25.738257259130478, 17.3..."
4,2차전지,"[-179.3514122008346, -68.59068633103743, 23.49..."


In [79]:
theme_vectors_df = theme_without_normal(news_vectors_df)

In [80]:
theme_vectors_df.head()

,themes,vectors
0,2차전지,"[-2019.3120858357288, -9628.890865209338, 4216..."
1,3D프린터,"[-5564.484228818386, -14353.711075838422, 1073..."
2,4대강,"[10274.747129786061, -8707.735842478229, 8529...."
3,5G,"[-6841.457318721339, -12318.28062159248, 5839...."
4,AR,"[-5471.329426370736, -7927.855614227825, -3063..."


In [81]:
theme_vectors_df.to_csv('theme_vectors_xnorm')

## Test

In [82]:
# news vectors without normalization

def vectorize_without_normal(news):
    news_words = news.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    token = sp(news)
    #stop_words = ['으로', '로도', '지만', '에서', '려는', '하다']
    #for word in token:
        #if len(word) == 1:
            #stop_words.append(word)
    final_tokens = [word for word in token if word not in stop_words]
    init_v = np.array([0.0]*v_dimension)
    for word in final_tokens:
        word_vectors = model.wv
        if word in word_vectors.vocab:
            v = model.wv[word]
            init_v = init_v + v
    return init_v

In [83]:
# test

def test(news):
    news_vec = vectorize_without_normal(news)
    result = []
    for theme in theme_vectors_df['vectors']:
        cosine = 1 - distance.cosine(theme, news_vec)
        result.append(cosine)
    new_df = pd.DataFrame(data=np.zeros([267,2]), columns=['Theme', 'Result'])
    new_df['Theme'] = theme_vectors_df['themes']
    new_df['Result'] = result

    new_df.sort_values('Result', ascending=False, inplace=True)
    return new_df
    

In [88]:
real_test = pd.read_csv("20200629_news_data.csv",index_col=0)

In [90]:
partial_test = real_test[0:100]
partial_test

,news
0,통신사 관계 없이 이용 가능 24개월 인수형 장기렌탈 서울 뉴시스 오동현 기자 KT...
1,20년 된 전자금융거래법 정비돼야 전자금융 모든 거래 총괄하게 되면 내년 비대면 상...
2,구내식당 모바일스루 서비스…7월 3곳 시범 적용 서울 뉴시스 오동현 기자 모바일 전...
3,소니가 프리미엄 디지털카메라로 분류한 RX100 제품군은 등장 초기에는 존재감이 분...
4,법원 행정처분 집행 정지 인용 지디넷코리아 손예술 기자 법원이 하나금융지주 함영주 ...
...,...
95,교보정보통신과 업무협약 교보정보통신이 클라우드 관리 기업 MSP 메가존클라우드와 손...
96,금융위원회 금융분야 마이데이터 포럼 개최 자본금·물적요건·사업계획 타당성 등 6개 ...
97,대전 생명공학연구원 주도 바이오융합클러스터 구축 나서 오송은 삼성제약·하얏트와 협업...
98,롯데렌탈과 장기렌탈 상품 출시 KT 직원들이 슈퍼 VR 장기 렌탈 상품을 소개하고 ...


In [91]:
lst = []
lst2 = []
for i in range(len(partial_test)):
    a = partial_test['news'][i]
    result = test(a)
    result = result.sort_values(by='Result', ascending=False).reset_index(drop=True)
    theme = result['Theme'][0]
    cosine = result['Result'][0]
    lst.append(theme)
    lst2.append(cosine)
    

In [93]:
final_data = pd.DataFrame(list(zip(lst, lst2)), columns =['Theme', 'Similarity']) 
final_data

,Theme,Similarity
0,VR,0.868391
1,FinTech,0.846018
2,무인기기,0.819095
3,이어폰,0.823447
4,김철민,0.762492
...,...,...
95,빅데이터,0.796844
96,데이터3법,0.808951
97,우주항공산업,0.875748
98,캐릭터상품,0.854310


In [94]:
our_result = pd.concat([partial_test,final_data],axis=1)
our_result

,news,Theme,Similarity
0,통신사 관계 없이 이용 가능 24개월 인수형 장기렌탈 서울 뉴시스 오동현 기자 KT...,VR,0.868391
1,20년 된 전자금융거래법 정비돼야 전자금융 모든 거래 총괄하게 되면 내년 비대면 상...,FinTech,0.846018
2,구내식당 모바일스루 서비스…7월 3곳 시범 적용 서울 뉴시스 오동현 기자 모바일 전...,무인기기,0.819095
3,소니가 프리미엄 디지털카메라로 분류한 RX100 제품군은 등장 초기에는 존재감이 분...,이어폰,0.823447
4,법원 행정처분 집행 정지 인용 지디넷코리아 손예술 기자 법원이 하나금융지주 함영주 ...,김철민,0.762492
...,...,...,...
95,교보정보통신과 업무협약 교보정보통신이 클라우드 관리 기업 MSP 메가존클라우드와 손...,빅데이터,0.796844
96,금융위원회 금융분야 마이데이터 포럼 개최 자본금·물적요건·사업계획 타당성 등 6개 ...,데이터3법,0.808951
97,대전 생명공학연구원 주도 바이오융합클러스터 구축 나서 오송은 삼성제약·하얏트와 협업...,우주항공산업,0.875748
98,롯데렌탈과 장기렌탈 상품 출시 KT 직원들이 슈퍼 VR 장기 렌탈 상품을 소개하고 ...,캐릭터상품,0.854310


In [95]:
cleansed = our_result[our_result['Similarity']>=0.8].reset_index(drop=True)
cleansed = cleansed[['Theme','news']]
cleansed

,Theme,news
0,VR,통신사 관계 없이 이용 가능 24개월 인수형 장기렌탈 서울 뉴시스 오동현 기자 KT...
1,FinTech,20년 된 전자금융거래법 정비돼야 전자금융 모든 거래 총괄하게 되면 내년 비대면 상...
2,무인기기,구내식당 모바일스루 서비스…7월 3곳 시범 적용 서울 뉴시스 오동현 기자 모바일 전...
3,이어폰,소니가 프리미엄 디지털카메라로 분류한 RX100 제품군은 등장 초기에는 존재감이 분...
4,클라우드,마포구 독거노인 모니터링 시스템 상세조회 화면. MS 제공 한국마이크로소프트 MS ...
...,...,...
63,김철민,간사는 같은 당 안호영 의원 지디넷코리아 박영민 기자 21대 국회 환경노동위원회 환...
64,데이터3법,금융위원회 금융분야 마이데이터 포럼 개최 자본금·물적요건·사업계획 타당성 등 6개 ...
65,우주항공산업,대전 생명공학연구원 주도 바이오융합클러스터 구축 나서 오송은 삼성제약·하얏트와 협업...
66,캐릭터상품,롯데렌탈과 장기렌탈 상품 출시 KT 직원들이 슈퍼 VR 장기 렌탈 상품을 소개하고 ...


In [98]:
cleansed.groupby('Theme').count().sort_values(by='news',ascending=False)

,news
Theme,
클라우드,8
김철민,4
스마트스터디,3
생체인식,3
캐릭터상품,3
이어폰,3
FinTech,2
전자결제,2
엔터테인먼트,2


In [86]:
result.head()

,Theme,Result
266,희토류,0.827587
102,불화수소,0.758883
234,통신장비,0.755397
254,항공기부품,0.752090
27,가스관사업,0.739626
